# 0. Meta

## 0.1. Packages

In [78]:
import pandas as pd
from datetime import datetime
import numpy as np
import os
import joblib

from sklearn.linear_model import LinearRegression

from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import KFold, cross_val_predict

## 0.2. Functions

# 1. Data Import

In [79]:
X_test = pd.read_csv("../data/processed/X_test.csv")
y_test = pd.read_csv("../data/processed/y_test.csv")
X_train = pd.read_csv("../data/processed/X_train.csv")
y_train = pd.read_csv("../data/processed/y_train.csv")

# 2. Model Training and Hyperparameter Tuning

## 2.1. Linear Regression

Train linear regression model and measure computation time.

In [80]:
lr_model = LinearRegression() 
lr_start = datetime.now()
lr_model.fit(X_train, y_train)
lr_pred = lr_model.predict(X_test)
lr_stop = datetime.now()
lr_delta = lr_stop - lr_start

Due to the log-transformation applied to the target variable y_train during data preprocessing to ensure normality and linearity, it is necessary to reverse this transformation on the predictions before assessing the model using metrics such as RMSE and R-squared.

In [81]:
y_test_log = np.log1p(y_test.to_numpy())

lr_pred_df = pd.DataFrame({'pred': lr_pred.flatten(), 'y_test': y_test_log.flatten()})

lr_pred_df['pred'] = np.expm1(lr_pred_df['pred'])
lr_pred_df['y_test'] = np.expm1(lr_pred_df['y_test'])

c:\Sync\03_projects\data_science\projects\autoscout24-car-price-prediction\envs\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: overflow encountered in expm1
  result = getattr(ufunc, method)(*inputs, **kwargs)


During the back-transformation, NaNs and Infs occurred for some rows. These entries are eliminated, and the indices of the removed rows are stored to identify the observations that led to these issues. Additionally, the total count and the relative proportion of dropped rows are computed.

In [82]:
index_before = lr_pred_df.index
rows_before = lr_pred_df.shape[0]

lr_pred_df = lr_pred_df.replace([np.inf, -np.inf], np.nan).dropna()

index_after = lr_pred_df.index
rows_after = lr_pred_df.shape[0]

removed_indices = index_before.difference(index_after)
removed_rows = rows_before - rows_after
percent_removed = (removed_rows / rows_before) * 100

removed_rows_X_test = X_test.iloc[removed_indices]
removed_rows_y_test = y_test.iloc[removed_indices]
removed_rows_df = pd.concat([removed_rows_X_test, removed_rows_y_test], axis=1)
removed_rows_df['pre_backtrans_pred'] = lr_pred.flatten()[removed_indices]
filtered_removed_rows_df = removed_rows_df.loc[:, (removed_rows_df != 0).any()]

print('Rows before removing NaNs and Infs:', rows_before)
print('Rows after removing NaNs and Infs:', rows_after)
print('Number of rows removed:', removed_rows)
print('Percentage of rows removed:', percent_removed, '%')

print("Problematic rows in X_test and y_test:")
filtered_removed_rows_df

Rows before removing NaNs and Infs: 9215
Rows after removing NaNs and Infs: 9199
Number of rows removed: 16
Percentage of rows removed: 0.17362995116657626 %
Problematic rows in X_test and y_test:


,mileage,offerType,hp,year,make_Audi,make_BMW,make_Chevrolet,make_Corvette,make_DS,make_Fiat,...,model_S7,model_SLC 250,model_XC70,fuel_Diesel,fuel_Electric,fuel_Gasoline,gear_Automatic,gear_Manual,price,pre_backtrans_pred
77,0.139248,0.50,0.538058,1.0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,83870,5.434110e+08
486,0.620149,0.00,0.483411,0.5,0,0,0,0,0,0,...,0,0,1,1,0,0,0,1,13470,1.714280e+08
824,0.406800,0.00,0.339380,0.5,0,0,0,0,1,0,...,0,0,0,0,0,1,0,1,8500,2.472846e+07
1032,0.283372,0.00,0.678830,0.7,0,1,0,0,0,0,...,0,0,0,0,0,1,1,0,35000,2.789572e+07
1424,0.076631,0.50,0.634355,1.0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,74890,3.837702e+08
1753,0.028231,0.00,0.379640,1.0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,22990,3.671505e+08
1983,0.430180,0.00,0.520298,0.5,0,0,0,0,0,0,...,0,0,0,1,0,0,1,0,19480,1.714280e+08
2468,0.126833,0.00,0.770530,0.8,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,86885,2.789572e+07
3623,0.398366,0.00,0.744444,0.3,1,0,0,0,0,0,...,1,0,0,0,0,1,1,0,39925,6.515272e+07
4405,0.020801,0.75,0.409779,1.0,0,0,0,0,0,1,...,0,0,0,0,1,0,1,0,33994,1.160684e+09


It appears that the linear regression model leads to extremely high predicted values for the problematic rows. Therefore, the back transformation fails, leading to infinite values. The full linear regression model therefore appears to be unsuitable. After removing the problematic rows, the evaluation metrics are calculated.  However, it should be noted that these can only be interpreted to a limited extent, as not all predictions are taken into account.

In [83]:
lr_r2 = r2_score(lr_pred_df['y_test'], lr_pred_df['pred'])
lr_r2_adj = 1 - (1 - lr_r2) * ((len(X_test) - 1) / (len(X_test) - len(X_test.columns) - 1))
lr_rmse = np.sqrt(mean_squared_error(lr_pred_df['y_test'], lr_pred_df['pred']))
lr_seconds = lr_delta.seconds + lr_delta.microseconds/1E6

lr_evaluation = pd.DataFrame({
    'model': ['lr'],
    'r2': [lr_r2],
    'r2_adj': [lr_r2_adj],
    'rmse': [lr_rmse],
    'seconds': [lr_seconds]
})

lr_evaluation

,model,r2,r2_adj,rmse,seconds
0,lr,0.914823,0.905329,5368.075742,3.8272


Save the trained model, model predictions and evaluation metrics.

In [84]:
joblib.dump(lr_model, '../models/models/lr_model.pkl')
lr_evaluation.to_csv(os.path.join("../models/evaluation/", "lr_evaluation.csv"), index=False)
lr_res = pd.DataFrame(lr_pred)
lr_res.index = X_test.index
lr_res.columns = ["prediction"]
lr_res.to_csv("../models/predictions/lr_prediction.csv")

## 2.2. Regularized Linear Regression

### 2.2.1. Lasso Regression

### 2.2.2. Ridge Regression

### 2.2.3. Elastic Net Regression

## 2.3. Gaussian Process Regression

## 2.4. Bayesian Linear Regression

## 2.5. Robust Regression

### 2.5.1. Huber Regression

### 2.5.2. Quantile Regression

### 2.5.3. RANSAC Regression

### 2.5.4. Theil Sen Regression

## 2.6. K-Nearest Neighbors Regression

## 2.7. Artificial Neural Networks

### 2.7.1. Multi-Layer Perceptron Regressor

## 2.8. Support Vector Regression

## 2.9. Decision Trees Regression

## 2.10. Ensemble

### 2.10.1. Ada Boost Regressor

### 2.10.2. Bagging Regressor

### 2.10.3. Extra Tree Regressor

### 2.10.4. Gradient Boosting Regressor

### 2.10.5. XGBoost Regressor

### 2.10.6. LightGBM Regressor

### 2.10.7. Random Forest Regressor

### 2.10.8. Extra Trees Regressor

### 2.10.9. Stacking Regressor

### 2.10.10. Voting Regressor

### 2.10.11. Histogram-based Gradient Boosting Regressor

## 2.10. Dimensionality-Reduced Regression